In [44]:
import requests
from bs4 import BeautifulSoup
import time

In [46]:
class Text:
    def __init__(self, title, author, content):
        self.title = title
        self.author = author
        self.content = content
    
    def get_title(self):
        return self.title
    
    def get_author(self):
        return self.author
    
    def get_content(self):
        return self.content



def get_response(url):
    return requests.get(url)

def get_page_url_list(start, end):
    page_url_list = []
    for i in range(start, end + 1):
        page_url_list.append("http://poemlove.co.kr/bbs/board.php?bo_table=tb01&page=%d" % (i))
    return page_url_list


def get_content_url_list(url):
    response = get_response(url)
    try:
        content_url_list = []
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            title_list = soup.select('#fboardlist > div.list-board > ul > li a')
            for title in title_list:
                href = title.attrs["href"]
                if("http://" in href):
                    content_url_list.append(href)
        return content_url_list
    except:
        print('Error Occured: get_content_url_list()')

def get_content_text(soup):
    title = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > div:nth-child(4) > div.view-content')
    return title.text

def get_author_text(soup):
    author = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > div:nth-child(3) > div > strong:nth-child(2)')
    return author.text

def get_title_text(soup):
    title = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > h1')
    return title.text

def get_content_data(url):
    response = get_response(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = get_title_text(soup)
    author = get_author_text(soup)
    content = get_content_text(soup)
    return Text(title,author,content)


In [47]:
def get_text_list():
    page_url_list = get_page_url_list(1, 3)

    content_url_list = []
    for page_url in page_url_list:
        content_url_list.extend(get_content_url_list(page_url))
        time.sleep(0.1)
    
    text_list = []
    for content_url in content_url_list:
        text_list.append(get_content_data(content_url))
        if(len(text_list) % 10 == 0):
            print("Get " + str(len(text_list)) + " data!") 
        time.sleep(0.2)

    print("Get " + str(len(text_list)) + " data!")    
    return text_list




In [48]:
text_list = get_text_list()

Get 10 data!
Get 20 data!
Get 30 data!
Get 40 data!
Get 50 data!
Get 60 data!
Get 70 data!
Get 78 data!


In [49]:
len(text_list)

78

In [56]:
text_list[9].get_title()

'\n\t\t\t가시고기 사랑\t\t'

In [57]:
text_list[9].get_author()

'도지현'

In [58]:
text_list[9].get_content()

'\n\t\t\t\t가시고기 사랑 / 藝香 도지현\r 주름투성이의 구릿빛 얼굴\r그래도 무엇이 그리 좋은지\r하회탈처럼 웃고 또 웃는다\r 마디마다 불거진 커다란 손은\r고목의 껍질에 갈퀴 같은데\r지문마저 닳아 보이지 않지만\r쉴 새 없이 움직이며 하루를 짓는다\r 자식을 위해서라면\r푸른 물속에서 입으로 품어\r자신을 헌신하여 자라나면\r온전한 성인으로 살게 하는\r가시고기와 같은 사랑으로\r 오늘도 아버지는 자식 생각에\r살점이 하나하나 떨어져 나가도\r그것을 스스로 최대의 기쁨이라 하며\r집으로 향한 발길은 가볍기만 하다.\t\t\t'